In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

# Análise I - Eficiência de Automóveis

Toda análise de dados fica muito mais simples se partirmos da necessidade de provar ou desprovar um hipótese. Essa hipótese serve como guia para nossos primeiros passos na análise, e, conforme trabalhamos o conjunto de dados, vamos complementando-a ou mesmo formulando novas hipóteses que queremos explorar.

Vamos começar com uma hipótese dada: *a eficiência (em termos de consumo de combustível) dos motores de automóveis aumentou ao longo dos anos*. 

In [ ]:
tb_veic = pd.read_csv("data/dados_veiculos.csv")
pattern = r"[^a-zA-Z0-9]"
tb_veic.columns = [re.sub(pattern, "_", column.lower()) for column in tb_veic.columns]
tb_veic.describe()

## Avaliação inicial

Nosso conjunto de dados contém 3 variáveis que medem a eficiência de consumo: `highway_mpg`, `city_mpg` e `combined_mpg`. Além disso temos o ano de cada automóvel. O primeiro passo pode ser verificar se a eficiência média por ano cresceu ou caiu.

In [ ]:
tb_agg_year = tb_veic.groupby(by="year").agg(
    mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
    mean_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="mean"),
    mean_combined_mpg=pd.NamedAgg(column="combined_mpg", aggfunc="mean"),
)
tb_agg_year

No código acima utilizamos um **named aggregation** para criar nossa tabela agregada. Poderíamos ter feito a mesma operação com uma agregação mais simples:

In [ ]:
# EXERCICIO

Vamos utilizar um gráfico simples para visualizar melhor essa evolução de eficiência:

In [ ]:
plt.plot(tb_agg_year)

### Investigando anomalias

Aparentemente a eficiência cresceu ao longo dos anos para as três variáveis. O gráfico acima tem duas peculiaridades: 

1. O primeiro ano apresenta uma eficiência muito menor que o segundo;
1. O último ano apresentou uma reversão na tendência de crescimento da eficiência.

Um problema de utilizar a **média** como agregador é a sensibilidade desta ao número de pontos que a compõe. Vamos analisar quanto automóveis temos em cada ano para determinar se está é a causa das anomalias acima.

In [ ]:
plt.plot(tb_veic.groupby('year')['make'].count())

Embora o número de automóveis apresente grande variação ano a ano, entre 1986 (terceiro ano) e 2015 (penúltimo ano), essa variação é continua (não temos saltos de ano a ano). Entretanto antes de 1986 e em 2016 temos discontinuidades (saltos). Isso pode indicar problemas na coleta de dados - algo muito comum nas *fronteiras* de um conjunto de dados.

Vamos limpar esses anos para tornar nossa análise mais assertiva.

In [ ]:
mask_ano = (tb_veic['year'] >= 1986) & (tb_veic['year'] <= 2015)
tb_veic = tb_veic[mask_ano]
tb_agg_year = tb_veic.groupby(by="year").agg(
    mean_city_mpg=pd.NamedAgg(column="city_mpg", aggfunc="mean"),
    mean_highway_mpg=pd.NamedAgg(column="highway_mpg", aggfunc="mean"),
    mean_combined_mpg=pd.NamedAgg(column="combined_mpg", aggfunc="mean"),
)
plt.plot(tb_agg_year)

## Aprofundando a análise

A análise acima conclui que a eficiência dos **automóveis** aumentou ao longo dos anos. No entanto, automóveis não são **motores**. Pode ser que os motores tenham se tornado menos potentes ao longo dos anos, e, por consequencia, os automóveis tornaram-se mais eficientes. 

Formulamos uma segunda hipótese: *os motores dos automóveis diminuiram em potência ao longo dos anos, levando a um aumento na eficiência geral*.

Qual variável podemos utilizar para medir a potência de um motor? Temos duas possíveis: o # de cilindros e a cilindrada (volume de deslocamento do motor). Podemos encontrar essas medidas nas variáveis `cylinders` e `engine_displacement`.

In [ ]:
tb_veic[['cylinders', 'engine_displacement']].describe()

Estas duas variáveis são numéricas, mas como discutimos antes, variáveis numéricas muitas vezes são **variáveis categóricas** (deveriam ser `strings`). Vamos olhar quantos valores distintos cada uma dessas variáveis assume em nosso conjunto de dados:

In [ ]:
tb_veic['engine_displacement'].value_counts()

In [ ]:
tb_veic['cylinders'].value_counts()

A variável `cylinders` parece ser categórica (o que faz sentido, afinal o # de cilindros é um inteiro, e em geral entre 4, 6 e 8). Já a variável `engine_displacement` parece ser mais continua (embora exista uma concentração de observações nos valores mais comuns como 2.0 e 3.0).

Vamos tratar a variável cilindros mantendo apenas os casos mais comuns (4, 6 e 8) e utilizar a função `pd.qcut()` para discretizar a variável `engine_displacement`.

In [ ]:
mask_cyl = tb_veic['cylinders'].isin([4, 6, 8])
tb_veic = tb_veic[mask_cyl].copy()
tb_veic['cat_ed'] = pd.qcut(tb_veic['engine_displacement'], 4, ['a', 'b', 'c', 'd'])

In [ ]:
plt.plot(tb_veic.loc[tb_veic['cylinders'] == 4, 'year'].value_counts().sort_index())

Parece que temos algo: o crescimento de eficiência observado a partir 2005 é diretamente correlato ao aumento no # de automóveis com motores v4. Vamos construir a mesma séries para os automóveis v6 e v8.

In [ ]:
tb_year_cyl4 = tb_veic.loc[tb_veic['cylinders'] == 4, 'year'].value_counts().sort_index().reset_index()
tb_year_cyl4.head()

In [ ]:
tb_year_cyl4.columns = ['year', 'n_4cyl']
tb_year_cyl4.head()

Agora vamos consolidar a criação das 3 séries em um celula só (clareza de código)...

In [ ]:
tb_year_cyl4 = tb_veic.loc[tb_veic['cylinders'] == 4, 'year'].value_counts().sort_index().reset_index()
tb_year_cyl4.columns = ['year', 'n_4cyl']
tb_year_cyl6 = tb_veic.loc[tb_veic['cylinders'] == 6, 'year'].value_counts().sort_index().reset_index()
tb_year_cyl6.columns = ['year', 'n_6cyl']
tb_year_cyl8 = tb_veic.loc[tb_veic['cylinders'] == 8, 'year'].value_counts().sort_index().reset_index()
tb_year_cyl8.columns = ['year', 'n_8cyl']


... e utilizar um merge para junta-las em uma tabela única:

In [ ]:
tb_year_cyl = tb_year_cyl4\
    .merge(tb_year_cyl6, on = 'year', how = 'left')\
    .merge(tb_year_cyl8, on = 'year', how = 'left')
tb_year_cyl.head()

In [ ]:
plt.plot(tb_year_cyl[['n_4cyl', 'n_6cyl', 'n_8cyl']])

Podemos fazer isso de uma maneira muito mais simples! Ainda mais se a nossa categoria tivesse muitos níveis. Como?

In [ ]:
# EXERCICIO

In [ ]:
tb_year_motor = tb_year_cyl.copy()
for cat in tb_veic['cat_ed'].unique():
    tb_year_ed = tb_veic.loc[tb_veic['cat_ed'] == cat, 'year'].value_counts().sort_index().reset_index()
    tb_year_ed.columns = ['year', f'n_ed{cat}']
    tb_year_motor = tb_year_motor.merge(tb_year_ed, on = 'year', how = 'left')
tb_year_motor

In [ ]:
plt.plot(tb_year_motor[['n_eda', 'n_edb', 'n_edc', 'n_edd']])

A evolução das categorias de cilindrada é muito semelhante à evolução do # de cilindros: inicialmente temos um aumento no tamanho dos motores e, após 20005, uma reversão dessa tendência com um aumento do número de automóveis com motores menores.

## Combinando Hipóteses

Até agora nossa análise nos mostrou duas coisas:

1. A eficiência de consumo dos **automóveis** cresceu ao longo dos anos, em especial de 2005 à 2015;
1. O tamanho dos motores incialmente cresce mas, após 2005, encolhe novamente.

Uma forma de combinar essas hipóteses é analisando a evolução da eficiência de consumo para cada uma das categorias que criamos de cilindradas.

In [ ]:
tb_eff_ed = tb_veic.groupby(['year', 'cat_ed'])['city_mpg'].mean().reset_index()
tb_eff_ed.head()

In [ ]:
tb_eff_ed.loc[tb_eff_ed['cat_ed'] == 'a', 'city_mpg']

In [ ]:
plt.plot(tb_eff_ed.loc[tb_eff_ed['cat_ed'] == 'a', 'city_mpg'])
plt.plot(tb_eff_ed.loc[tb_eff_ed['cat_ed'] == 'b', 'city_mpg'])
plt.plot(tb_eff_ed.loc[tb_eff_ed['cat_ed'] == 'c', 'city_mpg'])
plt.plot(tb_eff_ed.loc[tb_eff_ed['cat_ed'] == 'd', 'city_mpg'])

# Análise II - O que permite um animal sonhar?

A tabela `tb_sleep` contém informações muito interessantes que nos permitem estudar sob quas condições (ecológicas e constitucionais) um animal pode sonhar (ou em quais condições um animal pode sonhar mais). Vamos investigar juntos essas condições - buscando entender o impacto dos fatores ecológicos e constitucionais sobre a capacidade de um animal sonhar.

Para começar, precisamos determinar qual será a nossa variável resposta: qual variável (ou transformação de variável!) vamos utilizar para representar a capacidade de um animal sonhar. Depois precisamos formular um hipótese clara e objetiva sobre as diferentes variáveis (ou transformação de variáveis!) que queremos avaliar.

In [ ]:
tb_animals = pd.read_csv('http://www.statsci.org/data/general/sleep.txt', sep='\t')
tb_animals.head()